# Chapter 5. Monte Carlo Methods

 이번 장에서는 가치 함수들을 추정하고, 최적 정책을 발견하기 위한 학습 기법들을 다룬다. 단, complete knowledge of environment 를 가정하지 않고 optimal behavior 를 얻을 것이다.
> DP와 같이 모든 가능한 transition 값이 아니라 샘플링 된 transitions 값만으로도 충분히 강력한데, Monte Carlo methods 는 average sample returns 기반으로 강화 학습 문제들을 푼다.

 episotic task 들에 대해서만 다루며 하나의 episode 가 종료되었을 때에 value estimates 와 policies 값들이 변경된다. 즉, Monte Carlo methods 는 episode-by-episode 수준에서 점진적으로 학습이 이루어진다. (not step-by-step)
 
  가장 큰 차이점은 multiple states 가 존재한다는 것이고, 반환 값은 같은 에피소드 내에 다음에 발생할 states 들에 의존한다. 즉, 모든 행동 선택이 학습 도중에 발생한다는 것이고 이전 state 관점에서 보았을 때에는 nonstationary 한 환경이다. 이러한 nonstationarity 는 DP를 위한 GPI 를 통해 샘플의 반환값을 통해 가치 함수를 학습한다. 
  
   prediction problem -> policy improvement -> control problem using GPI

## 5.1 Monte Carlo Prediction
 경험을 통해서 추정하는 가장 명백한 방법은 state 의 방문 이후에 반환 값들의 평균값을 이용하는 것이여 이 아이디어는 모든 Monte Carlo methods 들의 기본이다.
 
 #### MC Methods
 ##### first-visit MC methods : state 에 처음 방문하는 경우들의 rewards 의 평균을 취함
 ##### every-visit MC methods : S의 이전 방문 경험에 대한 체크가 빠진 점만 차이점이다
> 어느 방법이든 방문 회수가 무한대로 가는 경우 가치함수는 수렴하게  되어 있으며, 대수의 법칙에 따라 estimates 값들의 averages 값은 expected value 에 수렴한다.
![first-visit](images/mc-first.png)

### Example 5.1: Blackjack
<br>
#### 기본적인 규칙
##### 1. 21을 초과하지 않고 카드 숫자의 합이 가능한 카드 조합을 찾는 문제
##### 2. 모든 얼굴 카드는 10으로 계산되며, 에이스는 1 또는 11로 계산 
##### 3. 각 플레이어들이 딜러와 독립적으로 경쟁하는 게임
##### 4. 게임은 딜러와 플레이어 모두에게 전달되는 두 장의 카드로 시작
##### 5. 딜러의 카드 중 하나가 앞면이 보이고 다른 쪽이 뒤집혀 있습니다
##### 6. 플레이어가 첫 번째 받은 2장이 A + 10 (10, J, Q, K) 이면 Blackjack or Natural 이라고 합니다.
##### 6-1. 이 때에 딜러가 Natural 이라면 Draw, 아니면 플레이어가 승리합니다.
##### 6-2. 반대의 경우 플레이어가 Natural 이 아니면, 순서대로 한 장식 더 가짐(Hit).
##### 6-3. 숫자가 큰 경우 거부(Stick)할 수 있으며 21을 넘어서는 경우 Bust 가 됩니다.
##### 7. Bust 경우 지게되고, 플레이어가 Stick 하면 딜러의 차례가 됩니다
##### 7-1. 딜러는 17점이 넘으면 Stick 하는 고정 전략을 따르고 그 이하는 Hit 합니다.
##### 8. 딜러가 Bust 하면 플레이어가 이기고, 그 외에는 21에 가까운 경우가 승리합니다.
<br>
#### MDP 문제로의 이해
##### 1. 블랙잭 게임은 episodic MDP 문제이며 매 경기는 하나의 episode 입니다
##### 2. 보상은 게임 종료 시에 승리 +1, 패배 -1, 무승부 0, discount factor 는 없으므로 $\gamma = 0$
##### 3. 플레이어의 action 은 hit or stick
##### 4. state 는 보여지는 플레이어의 카드와 딜러의 카드입니다
##### 5. 카드는 이미 처리 된 카드를 추적 할 수있는 이점이 없도록하기 위해 카드를 교체합니다 (즉 교체 포함)
##### 6. 플레이어는 세 가지 변수에 근거하여 결정을 내리며 총 200개의 상태가 만들어집니다.
##### 6-1. 플레이어 카드의 현재 숫자의 합 (12-21) [10]
##### 6-2. 딜러가 보여주는 한 장의 카드 (ace-10) [10]
##### 6-3.  사용 가능한 ace 카드의 보유 여부 [2]
> 플레이어 숫자의 합이 20 or 21 이라면 stick 아니면 hit 하는 정책을 생각해 보면,  Monte Carlo approach 를 통해 state-value function 을 찾기 위해 매 상태의 returns 의 averages 와 policy 를 이용하여 많은 blackjack 게임을 수행할 수 있다. 이를 통하여 estimates of state-value function 을 구할 수 있다.

![Estimates for states](images/figure5.1.png)
The estimates for states with a usable ace are less certain and less regular because these
states are less common. In any event, after 500,000 games the value function is very well
approximated.

 블랙 잭 작업에서 환경에 대한 완전한 지식을 가지고 있지만 값 기능을 계산하기 위해 DP 방법을 적용하는 것은 쉽지 않으며, DP 방법은 다음 이벤트의 분포를 필요로합니다. 특히 4 가지 인수 함수 p가 제공하는 환경 dynamics 가 필요하며 블랙잭을 위해 이것을 결정하는 것은 쉽지 않습니다.
> 플레이어의 합이 14이고 stick 을 선택한다고 가정할 때,  딜러의 카드 표시 기능에 +1의 보상으로 종료 할 확률은?

 DP가 적용되기 전에 모든 확률이 계산되어야하며 그러한 계산은 종종 복잡하고 오류가 발생하기 쉽습니다. 반대로 몬테카를로 메서드가 필요로하는 샘플 게임을 생성하는 것은 쉽습니다. Monte Carlo 방법은 에피소드 샘플링 만으로도 동작할 수 있는 능력은 환경의 dynamics 에 대한 완전한 지식을 갖추고있을 때조차도 큰 이점이 될 수 있습니다.
> 백업 다이어그램에 대한 아이디어를 Monte Carlo 알고리즘으로 일반화 할 수 있습니까?

 백업 다이어그램의 일반적인 개념은 업데이트 할 루트 노드를 맨 위에 표시하고 보상 및 예상 값이 업데이트에 기여하는 모든 전환 및 리프 노드를 표시하는 것입니다. v의 몬테카를로 추정에있어서, 루트는 상태 노드이고, 그 아래에는 특정 단일 에피소드를 따르는 전이 궤도의 전체가있다.
<br>
#### DP 와 MC 의 차이점
![backup](images/backup.png)
##### DP가 가능한 모든 전환을 표시, MC는 한 에피소드에서 샘플링 된 것만 보여줌
##### DP가 하나의 단계 전환 만 포함, MC는 에피소드가 끝날 때까지 계속된다.

<br>
#### MC 가 DP 보다 좋은 점
##### 1. MC 는 개별 state 에 대한 estimates 가  독립적이다.
#####  - 하나의 state 에 대한 estimate는 DP와 같이 다른 state 에 대한 estimate 을 기초로하지 않음
##### 2. MC는 bootstrap 하지 않으며,  단일 상태 값을 추정하는 계산 비용은 상태 수와 무관
#####  - 특히 one state or sub-state 의 value 만 얻기에 유용함
##### 3. 관심 state 에서 시작하여 많은 샘플 에피소드를 생성 할 수 있고, 다른 모든 것을 무시하고 이러한 상태에서만 평균을 반환

### Example 5.2: Soap Bubble
 아래 그림과 같이 폐 루프를 형성하는 와이어 프레임이 비눗물로 덩어리져 있고, 와이어 프레임에 그 가장자리에서 비누 표면 또는 기포를 형성한다고 가정 했을 때, 
 > 와이어 프레임의 기하학적 모양이 불규칙하지만 알려진 경우 표면의 모양을 어떻게 계산할 수 있습니까?
 ![bubble](images/bubble.png)

 이 모양에는 인접한 점이 작용하는 각 점의 총 힘이 0이되는 속성이 있습니다 (그렇지 않으면 모양이 변경됩니다). 이것은 어떤 점에서 표면의 높이가 그 점을 중심으로 작은 원의 점에서 높이의 평균임을 의미합니다. 또한 표면은 와이어 프레임과의 경계에서 만나야합니다.

 이런 종류의 문제에 대한 일반적인 접근법은 표면으로 덮인 영역 위에 그리드를 놓고 반복 계산에 의해 그리드 지점에서의 높이를 계산하는 것입니다. 그리드 경계에서의 점들은 와이어 프레임으로 가야하고 다른 모든 점은 네 개의 가장 가까운 이웃의 높이의 평균을 향해 조정됩니다.
> 이 프로세스는 DP의 반복적 인 정책 평가와 마찬가지로 반복되고 궁극적으로 원하는 표면에 가까운 근사치로 수렴됩니다.

 이는 몬테카를로 방법이 원래 설계된 문제의 종류와 유사합니다. 위에서 설명한 반복 계산 대신 표면에 서서 랜덤 보행을 하면서 경계에 도달 할 때까지 동일한 확률로 격자 점에서 인접한 격자 점으로 무작위로 스테핑하는 것을 상상해보십시오.

> 경계에서 높이의 예상 값은 시작점에서 원하는 표면의 높이에 가까운 근사치입니다 (실제로는 정확히 위에서 설명한 반복 방법에 의해 계산 된 값입니다). 따라서, 그 지점에서 시작된 많은 걷기의 경계 높이를 단순히 평균화함으로써 한 지점에서의 표면 높이를 근사 적으로 근사시킬 수 있습니다. 한 점의 값 또는 작은 점 집합에 관심이있는 경우이 Monte Carlo 방법은 지역 일관성에 기반한 반복 방법보다 훨씬 효과적 일 수 있습니다.


## 5.2 Monte Carlo Estimation of Action Values
> 여태까지 순차적 문제는 MDP 라는 방법론을 통해 해결할 수 있고, 이는 agent 의 상호작용을 통해 environment 의 dynamics 를 모방했다고 볼 수 있습니다. 즉 '모델링'이란 복잡한 environment 에서 agent 가 state 별로 action 을 취했을 때에 어떤 state 로 변화하고, 또한 얼마 만큼의 reward 를 얻을 수 있는지를 함수 혹은 방정식으로 추상화 한 것을 말합니다.

> 결국 여태까지 우리는 '순차적 의사결정 문제'를 해결하기 위해서 환경을 '벨만 방정식'으로 '모델링'하여 '동적 모델링' 기법 등을 이용하여 문제를 해결 하였다고 말할 수 있습니다.

  모델이 있다면 DP에서 다루었던 것처럼 미리 가치 함수가 계산 되어 있어서 한 단계 앞의 값을 알 수 있으면 '벨만 방정식'을 통하여 최적의 결과를 예측할 수 있으므로 'state values' 만으로도 policy 를 결정하는 것은 충분합니다. 하지만 모델이 없다면 'state values' 대신 'action values'를 추정하는 편이 유용할 것입니다.

> 이 표현을 다시 이해해 보면, 보상은 이미 알고 있으나 환경의 상태 전이를 알 수 없는 경우가 있을텐데요, '블랙잭'의 예제에서와 같이 딜러가 동일한 상태에서 행동할 수 있는 상황이 너무 다양하여 '가치 함수'를 미리 계산하는 것이 어렵기 때문에 'action values'를 이용하여 추정할 수 밖에 없다는 의미로 해석됩니다.

 즉, 모델이 없다면 'state values' 만으로는 정책을 결정하기는 어렵기 때문에 주요 목적 가운데 하나는 '몬테 카를로 메소드'를 통해 $q_*$ 함수를 예측하는 것이 목적이며, 우선 'action values'를 얻기 위한 _policy evaluation_ 문제를 해결해야 합니다.

  'policy evaluation' 문제는 정책 $\pi$를 따르는 상태 s 에서 액션 a 를 취했을 때에 예상되는 'return' 값인 $q_\pi(s,a)$를 추정 하는 것인데 (여태까지 벨만 방정식과 같은 내용은 언급되지 않았습니다.) '몬테 카를로 메소드'는 'state-action'을 다룬다는 점만 제외하면 'state valuse'와 동일하며, state-action 쌍에 대한 모든 경우에 대한 보상의 평균을 계산하는 'every-visit MC method'와 첫 번째 발생하는 경우에 대한 보상만 계산하는 'first-visit MC method' 두 가지가 있습니다. 이 두 메소드들은 무한히 반복을 거듭하는 경우에 'quadratically converge' 합니다.
  
  유일한 단점은 많은 'state-action' 쌍을 방문할 수 없다는 점인데요, 만약에 $\pi$가 'deterministic policy' 임을 가정했을 때에, 각 state 에서 여러 actions 가운데 하나의 action 에 대해서 여러개의 returns 를 관찰하게 될 수 있기 때문에, average 값에 대한 returns 가 없는 경우 경험을 통한 개선이 없을 수 있다는 점이다.


## 5.3 Monte Carlo Control

## 5.4 Monte Carlo Control without Exploring Start

## 5.5 Off-policy Prediction via Importance Sampling

## 5.6 Incremental Implementation

## 5.7 Off-policy Monte Carlo Control

## 5.8 *Discounting-aware Importance Sampling

## 5.9 *Per-decision Importance Sampling

## 5.10 Summary